In [140]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import scipy
from scipy.optimize import newton
from scipy.optimize import fixed_point

"""
use beta=0.95, 0.97
lambdah=0.41, 0.45, 0.49
"""


class gsc(object):
    def __init__(self, beta, lambdah):
        self.beta=beta
        self.lambdah=lambdah
        
    def lnwage(self, low):
        beta=self.beta
        temp1=(1-beta)/((1-low)*beta*(low+beta-1))
        temp2=(low+beta-1)/low
        return np.exp(temp1)*temp2

    """
    high=1-low
    llow=low+beta-1
    """
    def stage(self, low):
        beta=self.beta
        lambdah=self.lambdah
        temp1=(low+beta-1)*np.exp(-lambdah)
        temp2=np.sqrt(temp1**2+(1-low)*(1-low+2*(low+beta-1)))
        temp3=(temp1+temp2)/(1-low+2*(low+beta-1))
        return -np.log(temp3)/lambdah

    def wage(self, low):
        lambdah=self.lambdah
        return (np.exp(2*lambdah*self.stage(low))+1)/2


    def dwage(self, low):
        return self.lnwage(low)-self.wage(low)
    
    def nofriction(self):
        beta=self.beta
        lambdah=self.lambdah
        full_low1=fsolve(self.dwage, 0.3)
        full_wage1=wage(full_low1)
        full_stage1=stage(full_low1)
        full_learn1=(1-beta)/full_low1
        print("full wagehigh={}".format(full_wage1),"\n"
              "full critical stage={}".format(full_stage1), "\n"
             "full low skill={}".format(full_low1), "\n"
             "full learning intensity={}".format(full_learn1),
             "\n")

        full_low2=fsolve(self.dwage, 0.98)
        full_wage2=wage(full_low2)
        full_stage2=stage(full_low2)
        full_learn2=(1-beta)/full_low2
        print("full wagelow={}".format(full_wage2),"\n"
              "full critical stage={}".format(full_stage2), "\n"
             "full low skill={}".format(full_low2), "\n"
             "full learning intensity={}".format(full_learn2),
             "\n")
   
    def friction(self):
        beta=self.beta
        lambdah=self.lambdah
        def g(w):
            def f(x):
                return self.lnwage(x)-w
            return (fsolve(f, 0.1), fsolve(f, 0.8))
        def low(w):
            return (g(w)[0]+g(w)[1])/2
        def newwage(w):
            return self.wage(low(w))
        equi_wage=fixed_point(newwage, 1.16)
        equi_lownorth, equi_lowsouth=g(equi_wage)
        equi_stage=self.stage(low(equi_wage))
        equi_learnsouth=(1-beta)/equi_lowsouth
        equi_learnnorth=(1-beta)/equi_lownorth
        print("global wage={}".format(equi_wage),"\n"
              "global critical stage={}".format(equi_stage), "\n"
             "north low skill={}".format(equi_lownorth), "\n"
              "north learning intensity={}".format(equi_learnnorth), "\n"
             "south low skill={}".format(equi_lowsouth), "\n"
             "south learning intensity={}".format(equi_learnsouth))

In [142]:
g1=gsc(0.96, 0.4)
g1.nofriction()
g1.friction()

full wagehigh=[ 1.09600839] 
full critical stage=[ 0.21420323] 
full low skill=[ 0.2807231] 
full learning intensity=[ 0.14248917] 

full wagelow=[ 1.49261469] 
full critical stage=[ 0.83626155] 
full low skill=[ 0.88703829] 
full learning intensity=[ 0.04509388] 

global wage=[ 1.1469694] 
global critical stage=[ 0.32634355] 
north low skill=[ 0.14853479] 
north learning intensity=[ 0.26929717] 
south low skill=[ 0.66541242] 
south learning intensity=[ 0.06011309]
